### Where we see recommender systems
- Personalization is transforming our experience of the world
 - Information overload
 - Browsing is "history"
   - Need new ways to discover content
 - Personalization: Connects users & items
- Cases
 - Movie recommendations
   - Connect user with movies they may want to watch
 - Product recommendations
   - Recommendations combine global & session interests
 - Music recommendations
   - Recommendations from coherent & diverse sequence
 - Friend recommendations
   - Users and "items" are of the same "type"
 - Drug-target interactions
   - What drug should we "repurpose" for some disease?

### Building a recommender system
- Solution 0: Popularity
 - What are people viewing now? -> Rank by global popularity
 - Limitation -> *No personalization*


- Solution 1: Classification model
 - What's the probability I'll buy this product?
 - user info, Purchase history, product info.. -> `Classifier` -> Yes or No
 - Pros:
   - **Personalized:** Considers user info & purchase history
   - **Features can capture context:** Time of the day, what I just saw..
   - **Even handles limited user history:** Age of user..
 - Limitations:
   - Features may not be available
   - Often doesn't perform as well as collaborative filtering methods

### Solution 2: Co-occurrence matrix
- People who bought *diapers* also bought *baby wipes*
- **Matrix C:**
 - Store # users who bought both items **i & j**
   - ($\text{# items x # items}$) matrix
   - **Symmetric**: $\text{# purchasing i & j same as # for j & i} (C_{ij} = C_{ji})$ 
   
<img src="https://image.slidesharecdn.com/cikm-dunning-what-algorithms-really-matter-131031173329-phpapp01/95/which-algorithms-really-matter-45-638.jpg?cb=1383312258" width=400>

### Making recommendations using co-occurence
- User purchase *diapers*
 1. Look at *diapers* row of matrix
   - `[0 .... 4 .... 100 ....]`
   - (DVD .... pacifiers .... baby wipes ....)
 2. Recommend other items iwth largest counts
    - *baby wipes, milk, baby food, ...*

### Co-occurrence matrix must be normalized
- What if there are very popular items?
 - Popular baby item: *Pampers Swaddlers diapers*
 - For any baby item (e.g., *i=Sophie giraffe*)
   - large count $C_{ij}$ for *j=Pampers Swaddlers*
   - Sophie: `[0 .... 1 million .... ]`
   - Sophie: (DVD .... Pampers diapers ....)

- Result:
 - Drowns out ohter effects
 - Recommend based on popularity

### Normalize co-occurrences: Similarity matrix
- **Jaccard similarity:** normalizes by popularity
 - Who purchased **i and j** divided by who purchased **i or j**
 - $\frac{\text{# purchased i and j}}{\text{# purchased i or j}}$
<img src="https://image.slidesharecdn.com/slidesforcrowdmm2013-131024052827-phpapp01/95/crowdsourced-object-segmentation-with-a-game-24-638.jpg?cb=1386566914" width=400>

- Many other similarity metrics possible, e.g., **cosine similarity**


- **Limitations**
 - Only current page matters, **no history**
   - Recommend similar items to the one you bought
 - What if you purchased many items?
   - Want recommendations based on purchase history

### (Weighted) Average of purchased items
- User bought items *{diapers, milk}*
 - Compute user-specific score for each item **j** in inventory by combining similarities:
   - $\text{Score(user, baby wipes)} = w_1 \cdot S_{\text{baby wipes, diapers}} + w_2 S_{\text{baby wipes, milk}}$
 - Could also weight recent purchases more
 
- Sort $\text{Score(user, j)}$ and find item $j$ with highest similarity


- **Limitations**
 - Does **not** utilize:
    - context (e.g., time of day)
    - user features (e.g., age)
    - product features (e.g., baby vs. electronics)
  - Cold start problem
    - What if a new user or product arrives?

### Solution 3: Discovering hidden structure by matrix factorization
- Movie recommendation
 - Users watch movies and rate them
 - Each user only watches a few of the available movies
- Matrix completion problem
 - Rating = `[Movies x Users]` (**sparse matrix**)
 - Data: Users score some movies ($\text{u: user, v:movie}$)
   - $\text{Rating(u,v)}$ known for black cells
   - $\text{Rating(u,v)}$ unknown for white cells
 - Goal: Filling missing data
- Suppose we had $d$ topics for each user and movie
 - Describe *movie $v$* with topics $R_v$
   - How much is it *action, romance, darama ...*
   - $R_v$ = `[0.3, 0.01, 1.5, ...]`
 - Describe *user $u$* with topics $L_u$
   - How much she likes *action, romance, darama ...*
   - $L_u$ = `[2.5, 0, 0.8, ...]`
 - $\text{Rating}(u,v)$ is the product of the two vectors
   - $R_v$ = `[0.3, 0.01, 1.5, ...]`
   - $L_u$ = `[2.5, 0, 0.8, ...]`
     - $0.3 \cdot 2.5 + 0 + 1.5 \cdot 0.8 + ... = 7.2$
   - $L_u'$ = `[0, 3.5, 0.001, ...]`
     - $0 + 0.01 \cdot 3.5 + 1.5 \cdot 0.01 + ... = 0.8$
 - **Recommendations:** sort movies user hasn't watched by $\text{Rating}(u,v)$

#### Predictions in matrix form ( sparse )
- $\text{Rating(u, v)} = $
$
\begin{bmatrix}
u_1, v_1 &  u_1, v_2 & ... & u_1, v_j \\
u_2, v_1 &  u_2, v_2 & ... & u_2, v_j \\
\vdots &  \vdots & \ddots & \vdots \\
u_i, v_1 &  u_i, v_2 & ... & u_i, v_j \\
\end{bmatrix}
\approx
\begin{bmatrix}
L_1 \cdot R_1 &  L_1 \cdot R_2 & ... & L_1 \cdot R_v \\
L_2 \cdot R_1 &  L_2 \cdot R_2 & ... & L_2 \cdot R_v \\
\vdots &  \vdots & \ddots & \vdots \\
L_u \cdot R_1 &  L_u \cdot R_2 & ... & L_u \cdot R_v \\
\end{bmatrix}
$
$ \qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad~~~~~
=
\begin{bmatrix}
L_1 \\
L_2 \\
\vdots \\
L_i
\end{bmatrix}
\cdot
\begin{bmatrix}
R_1 & R_2 & \cdots & R_j
\end{bmatrix}
= <L_u, R_v>
$

- $\text{Rating(u, v)} = $ is real rating table
- $<L_u, R_v>$ is predicted rating table

#### Matrix factorization model: Discovering topics from data
- $\text{Rating(u, v)} ~ \approx ~ <L_u, R_v>$
 - $L, R^\prime$ are **Parameters of model**
 
 
- $RSS(L, R) = (\text{Rating(u, v)}~ - <L_u, R_v>)^2 + \text{[include all (u', v') pairs where Rating(u', v') are available]}$
 - Many efficient algorithms for factorization


- Only use observed values to estimate "topic" vectors $\hat{L_u}$ and $\hat{R_v}$
- Use estimated $\hat{L_u}$ and $\hat{R_v}$ for recommendations

#### Limitations of matrix factorization
- Cld-start problem
 - This model still cannot handle a new user or movie

### Combining features and discovered topics
- Features capture **context**
 - *Time of day, what I just saw, user info, past purchases, ...*
= Discovered topics from matrix factorization capture **groups of users** who behave similarly
 - *Women from Seattle who teach and have a baby*
 

- **Combine** to mitigate cold-start problem
 - Ratings for a new user from **features** only
 - As more information about user is discovered, matrix factorization **topics** become more relevant
 
#### Blending models ( Ensemble )
- Squeezing last bit of accuracy by blending models
- Netflix Prize 2006 - 2009
 - 100M ratings
 - 17,700 moveis
 - 480,189 users
 - Predict 3 million ratings to highest accuracy
 - **Winning team blended over 100 models**

### A performance metric for recommender systems

#### Why not use classification accuracy?
- Classification accuracy = *fraction of items correctly classified* (liked vs. not liked)
- Here, not interested in what a person does not like
- Rather, how quickly can we discover the relatively few liked items?
 - **(Partially) an imbalanced class problem**
- Highly risk, when we recommend not like items to user, rather than we could not


**Recall** = $\frac{\text{# liked & shown}}{\text{# liked}}$
 - How many itmes I like was shown in items I like
 
**Precision** = $\frac{\text{# liked & shown}}{\text{shown}}$
 - How many items I like was shown in items was recommended

#### Maximize recall:
- Just recommend everything -> Recall = 1
- Then, Rrecision = small or almost zero
- **Not good**

#### Optimal recommender
- Recall = 1
- Precision = 1
- The only things that I like was recommended!

#### Precision-recall curve
- Input: A specific recommender system
- Output: Algorithm-specific **precision-recall curve**


- To draw curve, vary threshold on # items recommended
 - For each setting, calculate the *precision and recall*
 
<img src="http://payload243.cargocollective.com/1/14/470136/7176781/precision-recall-eng-captioned_o.png" width=400>

#### Whici Algorithm is Best?
- For a given *precision*, want *recall* as large as possible (or vice versa)
- One metric: largest **area under the curve (AUC)**
- Another: set desired recall and maximaize precision (*precision at k*)

### Clustering and similarity ML block diagram
- `Training Data`( user, product, ratings table ) -> `Featrue extraction` -> $x$ ( user_id, product_id, age, gender, product description, ... )
- $x$ -> `ML model(matrix factorization)` ( $\hat{w} : \left\{ \hat{L_u}, \hat{R_v} \right\}, \hat{w_0}$ ) -> $\hat{y}$ ( predicted rating )
- `y` is actual ratings
- `Quality metric` -> `ML algorithm ( RSS )` -> ROC curve & AUC -> $\hat{w}$
 - loop, updating for maximize AUC or other metrics